# 从零开始实现线性回归
通过代码实现线性回归模型的构建、训练过程，进一步理解机器学习的整体流程

包括数据流水线、损失函数和小批量随机梯度下降优化器等
详细步骤如下:
1. 生成训练数据
2. 读取数据集（数据集批量分割）
3. 初始化模型参数
4. 定义模型
5. 定义损失函数
6. 定义优化器
7. 训练模型

In [1]:
import d2l
import torch
import random

### 1. 生成训练数据集

y = Xw + b + $\epsilon$

In [2]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.mv(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = torch.tensor(4.2)
num_examples = 1000
features, labels = synthetic_data(true_w, true_b, num_examples)
print(f'features shape: {features.shape}\nlabels shape: {labels.shape}')

features shape: torch.Size([1000, 2])
labels shape: torch.Size([1000, 1])


### 2. 读取数据集

In [3]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        end_index = min(i + batch_size, num_examples)
        batch_indices = indices[i:end_index]
        yield features[batch_indices], labels[batch_indices]

### 3. 初始化模型参数
从$N(\mu = 0, \sigma = 0.001)$的正态分布随机抽样初始化w 和 b

In [4]:
w = torch.normal(0, 0.001, true_w.shape, requires_grad=True)
b = torch.normal(0, 0.001, true_b.shape, requires_grad=True)
w, b

(tensor([-5.8748e-05,  3.5632e-05], requires_grad=True),
 tensor(0.0004, requires_grad=True))

### 4. 定义模型

In [5]:
def linreg(X, w, b):
    """线性回归模型"""
    return torch.mv(X, w) + b

### 5. 定义损失函数
标签值是数值型，采用 MSE 作为损失函数
$\frac{1}{2}(y - \hat{y})^2$

In [6]:
def squared_loss(y_pred, y_true):
    return (y_pred - y_true.reshape(y_pred.shape)) ** 2 / 2

squared_loss(torch.tensor(99), torch.tensor(100))

tensor(0.5000)

### 6. 定义优化算法

In [7]:
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

### 7. 训练模型
在每轮中完整遍历一次训练数据集，不断迭代从中获取小批量训练数据集，对于每个小批量训练数据集执行如下步骤：
* 获取小批量样本
* 获取样本预测
* 计算损失
* 反向传播，优化更新参数


In [8]:
lr = 0.03      # 学习率
num_epochs = 3 # 学习轮次
net = linreg   # 神经网络
lossfn = squared_loss # 损失函数
batch_size = 10 # 批次大小

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        y_pred = net(X, w, b)
        loss = lossfn(y_pred, y)
        # loss 的形状是[batch_size, 1]，不是一个标量
        # loss 中的所有元素想加，并以此计算关于[w, b]的梯度
        loss.sum().backward()
        # 使用参数的梯度更新参数
        sgd([w, b], lr, batch_size)
    # 完成一轮训练后，打印训练数据的拟合指标数值
    with torch.no_grad():
        train_loss = lossfn(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_loss.mean()):f}')
print(f'w: {w.detach().numpy()}, b: {b.detach().numpy()}')
print(f'w true: {true_w}, b true: {true_b}')

epoch 1, loss 0.032673
epoch 2, loss 0.000117
epoch 3, loss 0.000052
w: [ 1.9997011 -3.3994913], b: 4.199641704559326
w true: tensor([ 2.0000, -3.4000]), b true: 4.199999809265137
